# Constraint Optimization


**约束优化，也可称为“约束规划（Constraint Programming）”**，是指从非常多的候选方案中识别和选择可行解决方案的方法，其中问题可以根据任意约束进行建模。CP问题出现在许多应用工程学科中。

- **它基于可行性，找到可行的解决方案，而不完全是优化**（找到最优解决方案），并且更侧重于约束和变量而不是目标函数。事实上，CP问题甚至可能没有目标函数。使用CP是为了通过向问题添加约束，将一大堆可能的解决方案缩小到更易于管理的子集。

- 常见的约束优化问题包括员工排程、作业车间调度等。还有一些平时比较熟悉的算法问题，比如N皇后、数独求解等也可归纳到这一范畴中。


--------

关于求解器本身不再赘述，Ortools提供了几种解决CP问题的方法：
1. CP-SAT 求解器：使用SAT（满足性）方法的约束规划求解器，这是在原始CP求解器基础上迭代的新版本；
2. 原始的CP求解器。

P.S. 如果可以使用线性目标和线性约束对问题进行建模，那么可以考虑 **MPSolver**。ortools官网同样补充，遇到路径问题等，通常使用专门的车辆路径库来解决（即使它们可以用线性模型表示）。


---------

后面展示的代码是通过CP-SAT方法求约束规划问题的实战部分。由于官网最开始的示例有点简单，所以直接用了官网后面的一个案例：N皇后问题，以及我之前做过的数独求解的案例进行展示。


以N皇后为例，为了统计 $N \times N$ 的棋盘上放 N 个皇后，使得其同行同列同对角线不冲突，总共有多少种放法。我们要做的就是输出 $N \times N$ 的棋盘，使得“每一列都不同、每一行都不同、对角线也重复”。借助`AddAllDifferent` 方法，可以在遍历的过程中批量地加入约束，减少一些不必要的 `AddConstraint` 操作。

另一个案例是数独求解，我们需要输入一个 $9 \times 9$ 的棋盘，规定其中一些值为常量，剩下的值在1～9之间选取，选取的结果要满足“行、列、宫”约束。

> 在这种思路下，我们可以求解所有的变形数独问题。譬如，杀手数独可以通过增加“小宫内的数字和为某个定值”来实现，而“锯齿数独”则可以通过改变宫的编号，实现约束的灵活变化。



In [6]:
"""OR-Tools solution to the N-queens problem."""
import time
from ortools.sat.python import cp_model
class NQueenSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, queens):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__queens = queens
        self.__solution_count = 0
        self.__start_time = time.time()

    def solution_count(self):
        return self.__solution_count

    def on_solution_callback(self):
        current_time = time.time()
        print(
            f"Solution {self.__solution_count}, "
            f"time = {current_time - self.__start_time} s"
        )
        self.__solution_count += 1

        all_queens = range(len(self.__queens))
        for i in all_queens:
            for j in all_queens:
                if self.Value(self.__queens[j]) == i:
                    # There is a queen in column j, row i.
                    print("Q", end=" ")
                else:
                    print("_", end=" ")
            print()
        print()


def main(board_size):
    model = cp_model.CpModel() # 创建求解器

    # 创建变量，变量的下标就是其所处的列，变量的值就是其所在的行
    queens = [model.NewIntVar(0, \
        board_size - 1, f"x_{i}") for i in range(board_size)]

    # 约束条件
    model.AddAllDifferent(queens)

    # 对角线不能有冲突
    model.AddAllDifferent(queens[i] + i for i in range(board_size))
    model.AddAllDifferent(queens[i] - i for i in range(board_size))

    # 模型求解
    solver = cp_model.CpSolver()
    solution_printer = NQueenSolutionPrinter(queens)
    solver.parameters.enumerate_all_solutions = True
    solver.Solve(model, solution_printer)

    print("\nStatistics")
    print(f"  conflicts      : {solver.NumConflicts()}")
    print(f"  branches       : {solver.NumBranches()}")
    print(f"  wall time      : {solver.WallTime()} s")
    print(f"  solutions found: {solution_printer.solution_count()}")

if __name__ == "__main__":
    # 这里求解一个 11 皇后问题
    size = 11
    main(size)

Solution 0, time = 0.00818777084350586 s
_ _ _ _ _ Q _ _ _ _ _ 
Q _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ _ _ _ 
_ Q _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ Q _ _ _ 
_ _ Q _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ Q _ _ 
_ _ _ Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ Q _ 
_ _ _ _ Q _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ Q 

Solution 1, time = 0.008852005004882812 s
_ _ _ _ _ Q _ _ _ _ _ 
Q _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ Q _ _ _ _ 
_ _ _ _ _ _ _ _ _ Q _ 
_ _ _ _ _ _ _ Q _ _ _ 
_ _ Q _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ Q _ _ 
_ _ _ Q _ _ _ _ _ _ _ 
_ Q _ _ _ _ _ _ _ _ _ 
_ _ _ _ Q _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ Q 

Solution 2, time = 0.010380029678344727 s
_ _ _ _ _ _ Q _ _ _ _ 
_ _ _ _ _ _ _ _ _ Q _ 
_ _ _ Q _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ Q 
_ _ _ _ _ _ _ Q _ _ _ 
_ _ Q _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ Q _ _ 
_ Q _ _ _ _ _ _ _ _ _ 
_ _ _ _ Q _ _ _ _ _ _ 
Q _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ Q _ _ _ _ _ 

Solution 3, time = 0.011183023452758789 s
_ _ _ _ _ _ Q _ _ _ _ 
Q _ _ _ _ _ _ _ _ _ _ 
_ _ _ Q _ _ _ _ _ _ _ 
_ 